# Running Tune experiments with Nevergrad

This example demonstrates the usage of Nevergrad with Ray Tune.

We also combine the search algorithm based on `NevergradSearch` with `AsyncHyperBandScheduler` scheduler to demonstrate Ray Tune's modularity.

Background information:
- [ZOOpt website](https://github.com/polixir/ZOOpt)

Necessary requirements:
- `pip install ray[tune]`
- `pip install zoopt==0.4.1`

In [ ]:
# !pip install ray[tune]
!pip install zoopt==0.4.1

Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [ ]:
import time

import ray
from ray import tune
from ray.tune.suggest.zoopt import ZOOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
from zoopt import ValueType

Let's start by defining a simple evaluation function.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune two hyperparameters,
namely `width` and `height`, and `activation`.

In [ ]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 1
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

Next, our ``objective`` function takes a Tune ``config``, evaluates the `score` of your experiment in a training loop,
and uses `tune.report` to report the `score` back to Tune.

In [ ]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        tune.report(iterations=step, mean_loss=score)

In [ ]:
ray.init(configure_logging=False)

Now we construct the hyperparameter search space using `ConfigSpace`

Next we define the search algorithm built from `ZOOptSearch`, constrained  to a maximum of `8` concurrent trials.

In [ ]:
zoopt_search_config = {
    "parallel_num": 8
}
zoopt_search = ZOOptSearch(
    algo="Asracos",  # only support ASRacos currently
    budget=num_samples,
    **zoopt_search_config,
)

Furthermore, we define a `scheduler` to go along with our algorithm to showcase the modularity of Ray Tune.

In [ ]:
scheduler = AsyncHyperBandScheduler() 

The number of samples this Tune run is set to `1000`.
(you can decrease this if it takes too long on your machine).

In [ ]:
num_samples = 1000

In [ ]:
# If 1000 samples take too long, you can reduce this number.
# We override this number here for our smoke tests.
num_samples = 10

Finally, all that's left is to define a search space.

In [ ]:
search_config = {
    "steps": 100,
    "width": tune.randint(0, 10),
    "height": tune.quniform(-10, 10, 1e-2),
    "activation": tune.choice(["relu, tanh"])
}

And run the experiment.

In [ ]:
analysis = tune.run(
    objective,
    search_alg=algo,
    scheduler=scheduler,
    metric="mean_loss",
    mode="min",
    name="zoopt_exp",
    num_samples=num_samples,
    config=search_config,
)

Here are the hyperparamters found to minimize the mean loss of the defined objective.

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)

## Optional: passing the parameter space into the search algorithm

We can also pass the parameter space ourselves

In [ ]:
space = {
    # for continuous dimensions: (continuous, search_range, precision)
    "height": (ValueType.CONTINUOUS, [-10, 10], 1e-2),
    # for discrete dimensions: (discrete, search_range, has_order)
    "width": (ValueType.DISCRETE, [0, 10], True)
    # for grid dimensions: (grid, grid_list)
    "layers": (ValueType.GRID, [4, 8, 16])
}

In [ ]:
zoopt_search_config = {
    "parallel_num": 8
}
algo = ZOOptSearch(
    algo="Asracos",
    budget=num_samples,
    dim_dict=space,
    **zoopt_search_config,
)
scheduler = AsyncHyperBandScheduler() 

In [ ]:
analysis = tune.run(
    objective,
    search_alg=algo,
    scheduler=scheduler,
    metric="mean_loss",
    mode="min",
    name="nevergrad_exp",
    num_samples=num_samples,
    config=search_config,
)

In [ ]:
ray.shutdown()